In [26]:
ls

LICENSE  'Prepare Class I dataset as a CSV.ipynb'  README.md  class1.fasta.gz


In [27]:
import skbio

In [55]:
from collections import Counter, OrderedDict

rows = []


for r in skbio.read("class1.fasta.gz", format="fasta"):
    species_gene_allele = r.metadata["description"].split()[0]
    if species_gene_allele.startswith("H-2"):
        allele = species_gene_allele[-1]
        species_and_gene = species_gene_allele[:-1]
        gene = species_and_gene[-1]
        species = "H-2"
    else:
        parts = species_gene_allele.split("*")
        if len(parts) == 2:
            species_and_gene, allele = parts
            extra = ""
        else:
            species_and_gene, extra, allele = parts
        
        
        species = species_gene_allele.split("-")[0]
        gene = "-".join(species_and_gene.split("-")[1:]) + extra
        
    row = OrderedDict([
        ("species_prefix", species),
        ("gene", gene),
        ("allele", allele),
        ("seq", str(r)),
        ("id", r.metadata["id"]),
    ])
    rows.append(row)

df = pd.DataFrame.from_records(rows);
df.to_csv("class1_sequences.csv", index=False)
df

,species_prefix,gene,allele,seq,id
0,HLA,A,01:01,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,HLA:HLA00001
1,HLA,A,01:02,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFSTSVSRPGSGEPRF...,HLA:HLA00002
2,HLA,A,01:03,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,HLA:HLA00003
3,HLA,A,01:04,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,HLA:HLA00004
4,HLA,A,01:06,MAVMAPRTLLLLLSGALALTQTWAGSHSMRYFFTSVSRPGRGEPRF...,HLA:HLA01031
...,...,...,...,...,...
14988,H-2,K,k,MAPCMLLLLLAAALAPTQTRAGPHSLRYFHTAVSRPGLGKPRFISV...,sp|P04223|HA1K_MOUSE
14989,H-2,D,k,MGAMVPRTLLLLLAAALAPAQTRAGPHSLRYFETVVSRPGLGEPRF...,sp|P14426|HA13_MOUSE
14990,H-2,D,p,MAPRTLLLLLAAALAPTQTRAGPHSLRYFVTAVSRPGLGKPRYMEV...,sp|P14427|HA14_MOUSE
14991,H-2,K,q,PRFISVGYVDDTELVRFDSDAENPRYEPRARWMEQVEPEYWERNTQ...,sp|P14428|HA1Q_MOUSE


DeviceArray([-8.0666114e-09, -2.1927324e-08, -5.9604645e-08], dtype=float32)

In [81]:
from pepdata.amino_acid_alphabet import peptide_to_indices

In [91]:
from pepdata.amino_acid_alphabet import peptide_to_indices
encoded_sequences = df.seq.str.replace("?", "X").map(peptide_to_indices).values

In [109]:
glorot = jax.nn.initializers.glorot_uniform??

In [110]:
glorot = jax.nn.initializers.glorot_uniform()

In [115]:
w = glorot(key, shape=(21, 60))

In [116]:
w

DeviceArray([[-0.24441464,  0.08583663,  0.22082128, ..., -0.0889992 ,
              -0.19414742,  0.18772669],
             [-0.12343033,  0.27106592,  0.10427065, ...,  0.11683659,
               0.22325332,  0.24368568],
             [ 0.20049833, -0.11135293, -0.11803737, ...,  0.20692685,
               0.17882296,  0.18202947],
             ...,
             [ 0.1441197 , -0.23909573,  0.07927321, ..., -0.26075074,
               0.21366651,  0.08117434],
             [-0.15124331, -0.17003818,  0.17474356, ..., -0.2646241 ,
              -0.22696786, -0.07925101],
             [-0.21590526, -0.01780193, -0.03946471, ..., -0.10800146,
               0.00541897,  0.01096357]], dtype=float32)

In [118]:
def mse(y_pred, y_true):
    return ((y_pred-y_true)**2).mean()

In [135]:
A_idx = np.where((df.species_prefix == "HLA") & (df.gene == "A"))[0]

In [136]:
A_idx

DeviceArray([   0,    1,    2, ..., 3592, 3593, 3594], dtype=int32)

In [137]:
B_idx = np.where((df.species_prefix == "HLA")  & (df.gene == "B"))[0]

In [138]:
B_idx

DeviceArray([3595, 3596, 3597, ..., 8145, 8146, 8147], dtype=int32)

In [139]:
len(A_idx)

3595

In [140]:
len(B_idx)

4553

In [142]:
def forward(indices):
    return np.array([w[i] for i in indices])

In [144]:
jax.vmap(forward, in_axes=None, out_axes=)